In [85]:
import pandas as pd

In [86]:
nsw_co2 = pd.read_csv('./emissions/nsw_co2intensity.csv')
vic_co2 = pd.read_csv('./emissions/vic_co2intensity.csv')

In [87]:
vm_skus_tdp = pd.read_csv('./azurevms/azurevmskus_withtdp.csv')
vm_skus_tdp

,Unnamed: 0,skutype,skuname,corecount,ram,cpumaxcores,cputdp,memtdp,gpucount,gputdp,cputotaltdp,gputotaltdp,totaltdp
0,0,generalpurpose,Standard_DC1s_v2,1,4.0,8,95,1.5000,0.000,0,11.875000,0.0,13.375000
1,1,generalpurpose,Standard_DC2s_v2,2,8.0,8,95,3.0000,0.000,0,23.750000,0.0,26.750000
2,2,generalpurpose,Standard_DC4s_v2,4,16.0,8,95,6.0000,0.000,0,47.500000,0.0,53.500000
3,3,generalpurpose,Standard_DC8_v2,8,32.0,8,95,12.0000,0.000,0,95.000000,0.0,107.000000
4,4,generalpurpose,Standard_D1_v2,1,3.5,26,195,1.3125,0.000,0,7.500000,0.0,8.812500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,88,gpuacceleratedcompute,Standard_NC24sr_v3,24,448.0,14,135,168.0000,4.000,250,231.428571,1000.0,1399.428571
89,89,gpuacceleratedcompute,Standard_NV4as_v4,4,14.0,64,240,5.2500,0.125,300,15.000000,37.5,57.750000
90,90,gpuacceleratedcompute,Standard_NV8as_v4,8,28.0,64,240,10.5000,0.250,300,30.000000,75.0,115.500000
91,91,gpuacceleratedcompute,Standard_NV16as_v4,16,56.0,64,240,21.0000,0.500,300,60.000000,150.0,231.000000


In [115]:
# These are the inputs

desired_sku_list = [{'sku':'Standard_D3_v2','count':4}, {'sku':'Standard_F16s_v2', 'count':2}, {'sku':'Standard_NC12','count': 1}]
default_hours_month = 730.5
start_month = 10
average_util_pct = .30
desired_region = 'vic'
microsoft_pue_constant = 1.125

In [116]:
if(desired_region == 'nsw'):
    carbonintensity = nsw_co2[['date','CarbonIntensity']]
else:
    carbonintensity = vic_co2[['date','CarbonIntensity']]



In [117]:
# Filter skus
skus_list = [x['sku'] for x in desired_sku_list]
is_in_desired_skus = vm_skus_tdp['skuname'].isin(skus_list)

vm_desired_skus = vm_skus_tdp[is_in_desired_skus]
vm_desired_skus

,Unnamed: 0,skutype,skuname,corecount,ram,cpumaxcores,cputdp,memtdp,gpucount,gputdp,cputotaltdp,gputotaltdp,totaltdp
6,6,generalpurpose,Standard_D3_v2,4,14.0,26,195,5.25,0.0,0,30.0,0.0,35.25
39,39,computeoptimized,Standard_F16s_v2,16,32.0,26,195,12.00,0.0,0,120.0,0.0,132.00
82,82,gpuacceleratedcompute,Standard_NC12,12,112.0,12,135,42.00,2.0,300,135.0,600.0,777.00


In [118]:
# Append SKU count
vm_desired_skus['skucount'] =  vm_desired_skus['skuname'].map(lambda x: [y['count'] for y in desired_sku_list if y['sku'] == x].pop() )
vm_desired_skus['skucount']

/home/justinbarias/anaconda3/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


6     4
39    2
82    1
Name: skucount, dtype: int64

In [119]:
vm_desired_skus

,Unnamed: 0,skutype,skuname,corecount,ram,cpumaxcores,cputdp,memtdp,gpucount,gputdp,cputotaltdp,gputotaltdp,totaltdp,skucount
6,6,generalpurpose,Standard_D3_v2,4,14.0,26,195,5.25,0.0,0,30.0,0.0,35.25,4
39,39,computeoptimized,Standard_F16s_v2,16,32.0,26,195,12.00,0.0,0,120.0,0.0,132.00,2
82,82,gpuacceleratedcompute,Standard_NC12,12,112.0,12,135,42.00,2.0,300,135.0,600.0,777.00,1


In [120]:
vm_desired_skus['totalmonthlykwh'] = vm_desired_skus['totaltdp'] * vm_desired_skus['skucount'] * default_hours_month / 1000 * microsoft_pue_constant * average_util_pct
vm_desired_skus['totalmonthlykwh']

/home/justinbarias/anaconda3/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


6      34.762669
39     65.087550
82    191.564494
Name: totalmonthlykwh, dtype: float64

In [121]:
carbonintensity['CarbonIntensity']

0     796.317250
1     818.393298
2     840.086083
3     829.143371
4     856.159838
5     859.364697
6     851.856436
7     844.965079
8     803.094948
9     821.429837
10    786.749184
11    747.712950
12    779.815842
Name: CarbonIntensity, dtype: float64

In [122]:
# Generate CO2eq emissions per month
desired_skus_with_monthly_co2 = vm_desired_skus.apply(lambda x: carbonintensity['CarbonIntensity'].map(lambda y: y * x['totalmonthlykwh']), axis=1)

In [123]:
# Convert to KG
desired_skus_with_monthly_co2 = desired_skus_with_monthly_co2.apply(lambda x: x/1000)

In [124]:
desired_skus_with_monthly_co2

desired_skus_with_monthly_co2['TotalYearlyCO2InKg'] = desired_skus_with_monthly_co2.sum(axis=1)
desired_skus_with_monthly_co2['TotalYearlyCO2InKg']

6      369.704070
39     692.211875
82    2037.305404
Name: TotalYearlyCO2InKg, dtype: float64

In [125]:
desired_skus_with_monthly_co2

,0,1,2,3,4,5,6,7,8,9,10,11,12,TotalYearlyCO2InKg
6,27.682113,28.449535,29.203634,28.823236,29.762401,29.873810,29.612803,29.373241,27.917724,28.555093,27.349501,25.992498,27.108480,369.704070
39,51.830339,53.267215,54.679145,53.966911,55.725346,55.933943,55.445248,54.996707,52.271483,53.464856,51.207577,48.666804,50.756303,692.211875
82,152.546111,156.775098,160.930665,158.834430,164.009826,164.623763,163.185447,161.865308,153.844477,157.356791,150.713209,143.235253,149.385027,2037.305404


In [126]:
vm_desired_skus = vm_desired_skus.join(desired_skus_with_monthly_co2)

In [127]:
vm_desired_skus.head(5)

,Unnamed: 0,skutype,skuname,corecount,ram,cpumaxcores,cputdp,memtdp,gpucount,gputdp,...,4,5,6,7,8,9,10,11,12,TotalYearlyCO2InKg
6,6,generalpurpose,Standard_D3_v2,4,14.0,26,195,5.25,0.0,0,...,29.762401,29.873810,29.612803,29.373241,27.917724,28.555093,27.349501,25.992498,27.108480,369.704070
39,39,computeoptimized,Standard_F16s_v2,16,32.0,26,195,12.00,0.0,0,...,55.725346,55.933943,55.445248,54.996707,52.271483,53.464856,51.207577,48.666804,50.756303,692.211875
82,82,gpuacceleratedcompute,Standard_NC12,12,112.0,12,135,42.00,2.0,300,...,164.009826,164.623763,163.185447,161.865308,153.844477,157.356791,150.713209,143.235253,149.385027,2037.305404
